In [1]:
import networkx as nx
from collections import deque
import matplotlib.pyplot as plt

import random

def create_subgraph(graph, edge_fraction, q):
    selected_edges = set()  # 使用集合避免重复
    nodes_to_include = set()  # 用来存储需要包含的节点

    # 包括所有 in_tree 为 True 的边
    for u, v, attrs in graph.edges(data=True):
        if attrs.get('in_tree', False):
            selected_edges.add((u, v))

    # 计算应该随机选择的边的数量
    total_edges = len(graph.edges())
    sample_size = int(total_edges * q)  # 选择边数的1/8

    # 从所有边中随机选择部分边，注意排除已经选取的边
    all_edges = list(graph.edges(data=True))
    random_edges = random.sample(all_edges, min(sample_size, len(all_edges)))
    
    # 过滤掉已经选中的边
    filtered_random_edges = [(u, v, attrs) for u, v, attrs in random_edges 
                             if (u, v) not in selected_edges]

    # 准备按照 reduced cost 排序的边
    reduced_costs = []
    for u, v, attrs in filtered_random_edges:
        potential_u = graph.nodes[u]['potential']
        potential_v = graph.nodes[v]['potential']
        reduced_cost = attrs['cost'] - potential_u + potential_v
        reduced_costs.append((reduced_cost, u, v))

    # 按照 reduced cost 排序
    reduced_costs.sort()

    # 根据 edge_fraction 选择边
    needed_edges = int(len(graph.edges()) * edge_fraction)
    sorted_edges = [(u, v) for _, u, v in reduced_costs[:needed_edges]]

    # 将选择的边添加到 selected_edges
    selected_edges.update(sorted_edges)

    # 包含所有选中边的节点
    for u, v in selected_edges:
        nodes_to_include.update([u, v])

    # 创建子图
    sub_graph = nx.DiGraph()
    for node in nodes_to_include:
        sub_graph.add_node(node, potential=graph.nodes[node]['potential'])
    for u, v in selected_edges:
        attrs = graph.get_edge_data(u, v)
        sub_graph.add_edge(u, v, **attrs)

    return sub_graph



def update_potentials_from_tree(graph, root):
    """
    使用广度优先搜索更新生成树中的势能，考虑有向图中的反向边。
    """
    # 初始化势能
    for node in graph.nodes():
        graph.nodes[node]['potential'] = float('inf')
    graph.nodes[root]['potential'] = 0

    # 使用队列进行 BFS
    queue = deque([root])
    visited = set([root])
    i=0

    while queue:
        current = queue.popleft()
        i+=1
        

        # 遍历当前节点的所有邻居
        for neighbor in graph.neighbors(current):
            if graph[current][neighbor]['in_tree'] and neighbor not in visited:
                edge_cost = graph[current][neighbor]['cost']
                graph.nodes[neighbor]['potential'] = graph.nodes[current]['potential'] - edge_cost
                queue.append(neighbor)
                visited.add(neighbor)
        
        # 遍历所有指向当前节点的反向邻居（即所有以当前为目标的节点）
        for precursor in graph.predecessors(current):
            if graph[precursor][current]['in_tree'] and precursor not in visited:
                edge_cost = graph[precursor][current]['cost']
                graph.nodes[precursor]['potential'] = graph.nodes[current]['potential'] + edge_cost
                queue.append(precursor)
                visited.add(precursor)


def find_min_reduced_cost_arc(graph):
    """
    根据节点势能和边的费用，找到具有最小减少成本的弧。
    该函数假设每个节点在图的节点属性中都有一个 'potential' 属性。
    """
    min_reduced_cost = float('inf')
    min_arc = None

    for u, v in graph.edges():
        # 从图中获取节点 u 和 v 的势能
        potential_u = graph.nodes[u]['potential']
        potential_v = graph.nodes[v]['potential']

        # 计算减少成本
        reduced_cost = graph[u][v]['cost'] - potential_u + potential_v

        # 检查是否这是到目前为止的最小减少成本，并且流量小于容量
        if reduced_cost < min_reduced_cost and graph[u][v]['flow'] < graph[u][v]['capacity']:
            min_reduced_cost = reduced_cost
            min_arc = (u, v)
    #print(min_reduced_cost)  
    
    if(min_reduced_cost>=0):
        min_arc = None
    
    return min_arc

    


def find_cycle(graph, u,v):
    """
    在包含 'in_tree' 属性的生成树中找到添加进入弧后形成的闭环。
    graph: NetworkX 图对象
    entering_arc: 元组 (u, v)，表示新添加的边
    """
    
    # 创建生成树的子图
    tree_edges = [(x, y) for x, y in graph.edges if graph.edges[x, y].get('in_tree', False)]
    tree = nx.Graph()
    tree.add_edges_from(tree_edges)
    
    # 在生成树中找到从 v 到 u 的唯一路径
    try:
        path = nx.shortest_path(tree, source=v, target=u)
    except nx.NetworkXNoPath:
        return None  # 如果没有路径，返回 None
    
    # 将进入弧添加到路径末尾形成闭环
    cycle = path + [v]
    return cycle

def update_flows(graph, cycle, flow_update):
    """
    更新图中回路上的流。
    """
    for i in range(len(cycle) - 1):
        u, v = cycle[i], cycle[i + 1]
        if graph.has_edge(u, v):
            graph[u][v]['flow'] += flow_update
        else:
            graph[v][u]['flow'] -= flow_update

            

            

def update_tree(graph, entering_arc, exit_arc):
    """
    更新生成树，加入进入弧，移除退出弧
    """
    # 加入进入弧到生成树
    if entering_arc:
        u_enter, v_enter = entering_arc
        graph[u_enter][v_enter]['in_tree'] = True
    
    # 移除退出弧出生成树
    if exit_arc:
        u_exit, v_exit = exit_arc
        graph[u_exit][v_exit]['in_tree'] = False    
    
    
            
def network_simplex_method(graph):
    """
    网络单纯形法的主函数。
    graph: networkx的图，其中边的属性包括'cost'(费用), 'capacity'(容量) 和 'flow'(流量)。
    potentials: 节点的初始势能字典。
    """
    
    i=0
    while True:
        
        
        entering_arc = find_min_reduced_cost_arc(graph)
        i=i+1
        # 步骤2: 找到进入的弧
        
        
        #print(entering_arc)
        if entering_arc is None:
            print(i)
            break  # 已找到最优解

        # 步骤3: 找到回路
        cycle = find_cycle(graph, *entering_arc)
        #print(cycle)
        flow_update=float('inf')
        # 步骤4: 计算流量更新量
        for u, v in zip(cycle[:-1], cycle[1:]):
            if graph.has_edge(u, v) and flow_update>graph[u][v]['capacity'] - graph[u][v]['flow']:        
                flow_update =graph[u][v]['capacity'] - graph[u][v]['flow']
                exit_arc=u,v
            if graph.has_edge(v, u) and flow_update>graph[v][u]['flow']:
                flow_update =graph[v][u]['flow']
                exit_arc=v,u
        #print(flow_update)
        #print(exit_arc)                
       
       
        # 步骤5: 更新流量
        update_flows(graph, cycle, flow_update)
        
        if exit_arc!=entering_arc:
            update_tree(graph, entering_arc, exit_arc)
            # 重新计算势能 
            update_potentials_from_tree(graph, 1)

        
        
        

    # 返回流量最优分配的图和势能
    return graph

# 示例使用：
# 首先，创建一个网络图，设定各边的容量和费用，并初始化流量和节点势能



    
    

    


In [15]:
def calculate_total_cost(graph):
    total_cost = 0
    for u, v, data in graph.edges(data=True):
        total_cost += data['cost'] * data['flow']
        #print(total_cost)
       # print(data['flow'])
    return total_cost


def apply_network_simplex_to_subgraphs(graph):
    
    costs_over_iterations = []
    previous_total_cost = None
    j=0
    while True :
        j=j+1
        
        sub_graph=create_subgraph(graph,4/p,8/p)
    
    
        sub_graph=network_simplex_method(sub_graph)  
        
        # 更新原图的流量和势能
        for u, v, data in sub_graph.edges(data=True):
            if graph.has_edge(u, v):
                graph[u][v]['flow'] = data['flow']
                if 'in_tree' in data:
                    graph[u][v]['in_tree'] = data['in_tree']
        
        for n, data in sub_graph.nodes(data=True):
            if graph.has_node(n):
                graph.nodes[n]['potential'] = data.get('potential', graph.nodes[n].get('potential'))
                
        current_total_cost = calculate_total_cost(graph)
        #print(current_total_cost)
        costs_over_iterations.append(current_total_cost)
        if find_min_reduced_cost_arc(graph)== None :
            print(current_total_cost)
            break
    
    
    
        #if previous_total_cost is not None:
          # if current_total_cost  ==  previous_total_cost:
             #   print(f"Stopping iteration {j} due to minimal cost change.")
              #  break
        
      #  previous_total_cost = current_total_cost
        #print(previous_total_cost)
  
    return graph, costs_over_iterations



In [35]:
import numpy as py
import random

p=5
list1 = [random.randint(1, 1000) for _ in range(p)]
list2 = [random.randint(1, 1000) for _ in range(p)]

# 计算两个列表的总和
sum1 = sum(list1)
sum2 = sum(list2)

# 调整其中一个列表的元素，以确保两个列表的总和相等
while sum1 != sum2:
    if sum1 > sum2:
    # 如果list1的总和大于list2的总和，从list1中随机选择一个元素减小
        index = random.randint(0, p-1)
        reduction = random.randint(1, min(list1[index], sum1 - sum2))
        list1[index] -= reduction
        sum1 -= reduction
    else:
        # 如果list2的总和大于list1的总和，从list2中随机选择一个元素减小
        index = random.randint(0, p-1)
        reduction = random.randint(1, min(list2[index], sum2 - sum1))
        list2[index] -= reduction
        sum2 -= reduction

# 生成一个10x10的矩阵，元素范围在1到100之间（可以根据需要进行调整）
matrix = [[random.randint(1, 100) for _ in range(p)] for _ in range(p)]



In [36]:
import numpy as np
def northwest_corner_method(supply, demand):
    allocation = np.zeros((len(supply), len(demand)))
    i, j = 0, 0

    while i < len(supply) and j < len(demand):
        alloc = min(supply[i], demand[j])
        allocation[i][j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc

        if supply[i] == 0 and i < len(supply) :
            i += 1
        elif demand[j] == 0 and j < len(demand) :
            j += 1

    return allocation

# 使用西北角法求解
nw_allocation = northwest_corner_method(list1.copy(), list2.copy())
#print(nw_allocation)




In [40]:
import networkx as nx


#tree_graph = nx.DiGraph()
G = nx.DiGraph()
# 添加供应节点和需求节点

for i in range(p):
    G.add_node(i, potential=float('inf'))  
    

for j in range(p):
    G.add_node(p+j, potential=float('inf'))

count=0
# 添加边及其成本
for i, row in enumerate(matrix):
    for j, weight in enumerate(row):
        if nw_allocation[i][j] > 0 :
            G.add_edge(i, p + j, cost=weight, capacity=min(list1[i], list2[j]),flow=nw_allocation[i][j],in_tree=True)
            print(i,j)
            #tree_graph.add_edge(u, v, **data)
            count+=1
        else :
            G.add_edge(i, p + j, cost=weight, capacity=min(list1[i], list2[j]),flow=nw_allocation[i][j],in_tree=False)
            #print(i,j)

update_potentials_from_tree(G, 0)
print(G)
print(nw_allocation)
#node_info = G.nodes[980]
#print("Node 1's Information:", node_info)


0 0
0 1
1 1
1 2
2 2
2 3
2 4
3 4
4 4
DiGraph with 10 nodes and 25 edges
[[212. 142.   0.   0.   0.]
 [  0. 172. 576.   0.   0.]
 [  0.   0. 239. 129.  25.]
 [  0.   0.   0.   0. 372.]
 [  0.   0.   0.   0. 404.]]


In [21]:
# 执行网络单纯形法
optimized_graph,costs_over_iterations= apply_network_simplex_to_subgraphs(G)



0
(96, 132)
92
0
(10, 193)
101
0
(80, 125)
107
0
(51, 193)
137
0
(98, 107)
146
0
(66, 193)
95
0
(89, 128)
102
0
(71, 163)
83
0
(98, 138)
60
0
(57, 174)
48
0
(1, 193)
59
0
(44, 114)
47
0
(75, 184)
41
0
(57, 105)
28
0
(13, 107)
35
0
(93, 119)
34
0
(84, 125)
26
0
(44, 116)
24
0
(5, 106)
30
0
(8, 167)
22
0
(8, 170)
37
0
(13, 122)
44
0
(94, 181)
31
0
(38, 152)
29
0
(8, 184)
19
0
(38, 157)
25
0
(94, 103)
21
0
(78, 198)
19
0
(23, 176)
17
0
(24, 152)
22
0
(46, 162)
14
0
(4, 165)
16
0
(32, 164)
8
0
(92, 162)
20
0
(20, 175)
23
0
(11, 187)
7
0
(27, 142)
8
0
(65, 147)
5
0
(54, 118)
19
0
(17, 161)
7
0
(35, 193)
6
0
(65, 167)
10
0
(9, 133)
9
0
(20, 122)
8
0
(93, 190)
12
0
(76, 166)
11
0
(67, 122)
7
0
(59, 106)
9
0
(80, 162)
4
0
(20, 162)
6
0
(8, 139)
10
0
(4, 185)
4
0
(89, 131)
11
0
(98, 106)
15
0
(4, 100)
12
0
(90, 108)
10
0
(93, 162)
6
0
(15, 162)
17
0
(66, 131)
9
0
(35, 124)
9
0
(46, 165)
7
0
(88, 100)
13
0
(27, 169)
13
0
(7, 121)
9
0
(30, 134)
12
0
(4, 190)
5
0
(12, 134)
2
0
None
1
0
(3, 151)
7


In [33]:
graph=network_simplex_method(G)
x=calculate_total_cost(graph)
print(x)

0
(63, 102)
832
538411.0


In [34]:
import networkx as nx
import time


def transportation_problem(costs, supply, demand):
    """
    使用网络单纯形法解决运输问题。
    costs: 运输成本矩阵，其中costs[i][j]表示从i到j的单位运输成本。
    supply: 各个供应点的供应量列表。
    demand: 各个需求点的需求量列表。
    """
    assert len(costs) == len(supply), "供应点数量和成本矩阵维度不匹配"
    assert len(costs[0]) == len(demand), "需求点数量和成本矩阵维度不匹配"

    # 创建网络图
    G = nx.DiGraph()

    # 添加供应节点和需求节点
    for i, s in enumerate(supply):
        G.add_node(f'Source_{i}', demand=-s)  # 供应量为负需求
    for j, d in enumerate(demand):
        G.add_node(f'Target_{j}', demand=d)

    # 添加边及其成本
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            G.add_edge(f'Source_{i}', f'Target_{j}', weight=cost, capacity=demand[j])

    # 使用网络单纯形法找到最小成本流
    flow_dict = nx.min_cost_flow(G)

    return flow_dict

flow = transportation_problem(matrix, list1, list2)

current=0
for source, targets in flow.items():
    for target, value in targets.items():
        if value > 0:
            source_index = int(source.split('_')[1])  # 从 "Source_0" 中提取 0
            target_index = int(target.split('_')[1])  # 从 "Target_1" 中提取 1
            current += value * matrix[source_index][target_index]

print("Total cost:", current)

Total cost: 113764
